In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [ ]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
##Классификация rnn, построить эмбеддинги

Выберем новости

In [ ]:
categories = ['alt.atheism', 'talk.religion.misc',
              'comp.graphics', 'sci.space']

#загрузка тренировочных данных
newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=categories)

#тестовые данные
newsgroups_test = fetch_20newsgroups(subset='test',
                                      categories=categories)
newsgroups_train.filenames.shape

(2034,)

In [ ]:
import nltk, string
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Очистим датасет от стоп слов, знаков препинания и поделим на токены

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

#обработка текста, удаление стоп слов, знаков пунктуации, лемматизация
def preprocess_text(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    processed_text = ' '.join(tokens)

    return processed_text
newsgroups_train_preprocess = [preprocess_text(text) for text in newsgroups_train.data]
newsgroups_test_preprocess = [preprocess_text(text) for text in newsgroups_test.data]
newsgroups_train_preprocess[0]

"rych festival.ed.ac.uk r hawkes subject 3d texture rule go line 21 hi 've noticed save model mapping plane positioned carefully .3ds file reload restarting 3d given default position orientation save .prj file positions/orientation preserved anyone know information stored .3ds file nothing explicitly said manual saving texture rule .prj file 'd like able read texture rule information anyone format .prj file .cel file format available somewhere rych ====================================================================== rycharde hawkes email rych festival.ed.ac.uk virtual environment laboratory dept psychology tel +44 31 650 3426 univ edinburgh fax +44 31 667 0150 ======================================================================"

In [ ]:
newsgroups_train.target

array([1, 3, 2, ..., 1, 0, 1])

Выполним векторизацию текста

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=2000)  # Уменьшили max_features до 1000

X_train_tfidf = tfidf_vectorizer.fit_transform(newsgroups_train_preprocess)

X_test_tfidf = tfidf_vectorizer.transform(newsgroups_test_preprocess)

RNN модель

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.layers import Bidirectional, SpatialDropout1D, Dropout

In [ ]:
vocab_size = len(tfidf_vectorizer.get_feature_names_out())

max_sequence_length = 2000

num_classes = len(newsgroups_train.target)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=256, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_tfidf.toarray(), newsgroups_train.target, epochs=5, batch_size=64)

loss, accuracy = model.evaluate(X_test_tfidf.toarray(), newsgroups_test.target)
print(f'Test accuracy: {accuracy * 100:.2f}%')

Epoch 1/5
32/32 [==============================] - 46s 1s/step - loss: 3.3182 - accuracy: 0.2345
Epoch 2/5
32/32 [==============================] - 38s 1s/step - loss: 1.6046 - accuracy: 0.2557
Epoch 3/5
32/32 [==============================] - 39s 1s/step - loss: 1.4607 - accuracy: 0.2650
Epoch 4/5
32/32 [==============================] - 38s 1s/step - loss: 1.4252 - accuracy: 0.2768
Epoch 5/5
43/43 [==============================] - 14s 298ms/step - loss: 1.3732 - accuracy: 0.2912
Test accuracy: 29.12%


Метрики получились низкие, это говорит либо о плохой предобработке, либо о плохой модели. Решить данную проблему можно изменяя архитектуру модели, добавляяя больше слоев или используя иные функции активации. Также можно попробовать изменить предобработку датасета

Задача 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_twitter_ebbeddings = KeyedVectors.load_word2vec_format("/content/drive/My Drive/Colab Notebooks/glove.twitter.27B.100d.txt", binary=False)

In [ ]:
newsgroups = fetch_20newsgroups(subset='all')
valid_words = [word for text in newsgroups.data for word in text.split() if word in wv_twitter_ebbeddings]
print(len(valid_words))
word_embeddings_generator = (wv_twitter_ebbeddings[word] for word in valid_words if word in wv_twitter_ebbeddings)
word_embeddings = list(word_embeddings_generator)

3266957


In [ ]:
word_to_find = "drive"  # Замените на слово, которое вы хотите найти

if word_to_find in wv_twitter_ebbeddings:
    similar_words = wv_twitter_ebbeddings.similar_by_word(word_to_find, topn=10)
    for word, score in similar_words:
        print(f"Слово: {word}, Схожесть: {score}")
else:
    print(f"Слово '{word_to_find}' отсутствует в эмбеддингах.")

Слово: run, Схожесть: 0.8035754561424255
Слово: driving, Схожесть: 0.7964431047439575
Слово: walk, Схожесть: 0.7523223161697388
Слово: ride, Схожесть: 0.7445083260536194
Слово: road, Схожесть: 0.7376804351806641
Слово: truck, Схожесть: 0.7369340062141418
Слово: way, Схожесть: 0.7359057664871216
Слово: drives, Схожесть: 0.726816713809967
Слово: speed, Схожесть: 0.7257543206214905
Слово: car, Схожесть: 0.7253392338752747


In [ ]:
wv_twitter_ebbeddings['drive']

array([ 1.1290e-01, -4.2279e-02,  7.9799e-01, -9.0015e-01,  4.9171e-01,
        3.1423e-01,  4.4565e-01,  4.5166e-01,  4.7258e-01,  3.1820e-01,
        1.5646e-01,  2.1389e-01, -3.6214e+00,  1.6312e-01, -2.6417e-01,
       -5.7310e-01,  1.0452e-01,  1.5858e-01, -5.0048e-01, -4.7867e-01,
       -2.4728e-01, -6.3285e-01,  9.1945e-04, -6.4473e-02,  3.1435e-01,
        1.3471e-01, -4.4049e-01,  5.2006e-01, -2.6491e-02, -1.7642e-01,
       -2.8870e-02,  2.1734e-01, -3.9332e-02,  4.3349e-01, -8.6976e-02,
       -2.5281e-01,  1.9447e-01, -5.4666e-01, -2.7647e-01,  1.6736e-01,
       -8.7234e-01,  1.5688e+00, -6.0826e-01, -3.2705e-01,  1.9757e-01,
        2.1602e-01,  4.9319e-01,  4.1488e-02, -3.8608e-01, -4.3826e-02,
        1.8597e-01, -9.2721e-02, -6.3279e-01,  4.7154e-01, -1.6383e-01,
       -5.2938e-01, -9.9055e-02,  2.1034e-02, -2.6999e-01, -1.5535e-01,
        5.4690e-01,  5.6305e-01,  1.1684e-01,  1.1194e-01,  4.1658e-01,
        1.3018e-02, -2.5333e-01, -1.7507e-01,  3.1544e-01, -3.03

In [ ]:
wv_twitter_ebbeddings['run']

array([-3.7545e-02,  1.1898e-01,  2.6445e-01, -7.6977e-01,  8.6135e-02,
        4.9159e-01,  6.5186e-01,  3.3472e-01, -2.8600e-01,  1.8043e-01,
        4.3625e-01,  2.5313e-01, -3.7646e+00,  3.6397e-01, -6.4090e-01,
       -4.9278e-01, -3.2588e-01, -8.0788e-02, -5.5677e-01, -2.4818e-01,
       -2.6804e-02, -1.4919e-01, -6.3853e-02, -3.5695e-01,  2.2165e-01,
        2.7029e-02, -1.9695e-01,  1.5099e-01, -1.1226e-01,  1.5583e-01,
        2.1070e-01,  1.4041e-01, -2.9264e-01,  5.6477e-01,  2.0825e-03,
        3.1600e-01,  1.8472e-01, -4.9405e-01, -3.3496e-01,  2.9211e-01,
       -5.4029e-01,  8.7934e-01, -3.9010e-01,  6.5981e-02,  3.6901e-01,
       -2.8464e-02,  1.5208e-01,  7.1456e-01,  4.8595e-01,  2.2912e-01,
       -2.4792e-01, -2.3986e-01,  5.0812e-01,  5.0515e-01, -3.8752e-01,
       -2.4902e-01,  1.7928e-02,  5.8749e-02, -3.0073e-01, -2.6155e-02,
        5.7714e-03, -1.3286e-01,  4.3251e-04, -1.3966e-01,  3.6019e-01,
       -4.0055e-01, -1.2270e-01, -4.3459e-02,  1.2906e-01, -2.94

In [ ]:
wv_twitter_ebbeddings['walk']

array([-3.4195e-01, -1.7946e-01,  6.3669e-01, -7.6787e-01,  2.3682e-04,
        5.2308e-01,  5.1993e-01,  5.5501e-01, -4.3731e-01, -4.8372e-02,
        1.2470e-01,  1.4680e-01, -4.1649e+00,  4.4308e-01, -1.1076e+00,
        2.6056e-01, -4.4285e-01, -5.5069e-02, -4.8600e-01, -2.1142e-01,
        2.4941e-01, -4.1828e-01, -2.5149e-01, -5.7975e-01,  5.6504e-01,
        3.6682e-01, -9.8677e-02,  1.3879e-01, -4.0591e-01,  1.0388e-01,
        2.2349e-01,  1.3590e-01,  5.1235e-01,  4.1220e-01, -1.5530e-01,
       -2.1991e-01, -2.3850e-01,  1.2786e-01,  1.3052e-01,  5.7099e-01,
       -2.5087e-01,  4.7097e-01, -6.5942e-02,  6.2348e-02,  4.9478e-01,
        5.6009e-01,  7.0259e-01,  6.6891e-01,  1.6475e-01,  3.7009e-01,
       -8.5258e-03, -2.9449e-01,  2.7545e-01, -9.4577e-02, -5.8851e-01,
       -4.7611e-01, -3.8746e-01,  3.7393e-01,  1.3773e-01, -1.4819e-01,
        6.9004e-01,  3.0588e-01,  1.3439e-01, -5.0925e-01, -6.9567e-02,
       -6.1202e-01, -4.7589e-03, -2.5082e-01,  2.3496e-02,  1.11

В данном задании были отобраны слова из датасета, которые встречаются в предобученных эмбэддингах и найдены наиболее похожие по смыслу слова:
1) Успешно отфильтрованы слова из текстовых данных исходя из наличия их в предобученных эмбеддингах Twitter GloVe. Этот шаг позволит работать только с подходящими словами, улучшая качество анализа.

2) Построены эмбеддинги для отобранных слов и провели анализ их близости к другим словам. Это позволяет находить семантически близкие слова и определять степень их схожести.

3) Реализована функциональность поиска близких слов к заданному. Это может быть полезно, например, для нахождения синонимов или ассоциированных слов в текстовых данных.